# Generating surface plots

_Written by Lori Garzio, March 10, 2025_

This tutorial outlines methods for generating pcolormesh surface maps of variables grabbed from the Rutgers University Weather Research and Forecasting Model (RU-WRF 4.1) from the [RUWRF THREDDs server](http://tds.marine.rutgers.edu/thredds/cool/ruwrf/catalog.html) to compare to the analyses described in Golbazi et al 2022 [DOI 10.1088/1748-9326/ac6e49](https://iopscience.iop.org/article/10.1088/1748-9326/ac6e49/meta).

Using the [wrf_data_wrangler_grid.py](https://github.com/rucool/wind-science/blob/master/rmi-ocean-mixing/wrf_data_wrangler_grid.py) script, we grabbed model output from the summer of 2022 (June - August) for the control model version [1km_ctrl](https://tds.marine.rutgers.edu/thredds/catalog/cool/ruwrf/wrf_4_1_1km_ctrl_processed/catalog.html), a 1km model domain without simulated turbines, as well as data from the wind farm model version [1km\_wf2km\_nyb](https://tds.marine.rutgers.edu/thredds/catalog/cool/ruwrf/case_studies/wrf_4_1_1km_wf2km_upwellingctrl_processed/catalog.html), a 1km model domain with simulated 15 MW turbines spaced 2km apart and filling all wind energy lease areas in the New York Bight as of early 2023. Monthly files were generated for the following variables and heights (if applicable):

	- variables: T2, TEMP, U, V, U10, V10, UST, Q2, TKE_PBL, HFX, TSK
	- heights: surface, 120m, 160m, 200m, 300m

After grabbing all of the model output, we generated summer 2022 averages using [data_averages.py](https://github.com/rucool/wind-science/blob/master/rmi-ocean-mixing/data_averages.py). Splitting the process into these two steps was necessary because of the length of time and computer resources it takes to grab this large volume of model output. The resulting files containing the summer 2022 averages for each variable can be downloaded [here](https://marine.rutgers.edu/~lgarzio/rmi_ocean_mixing/) if you want to skip the data wrangling and averaging process.

In [43]:
# import required packages
import numpy as np
import os
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
import cmocean as cmo
from math import ceil
import cool_maps.plot as cplt
plt.rcParams.update({'font.size': 12})  # all font sizes are 12 unless otherwise specified

First, specify the locations of the NetCDF files containing the model output (control and turbines) you want to plot, as well as a location to save all of the images.

In [44]:
file_dir = '/Users/garzio/Documents/rucool/rmi-ocean-mixing/files/1km_wf2km_nyb'
file_dir_control = '/Users/garzio/Documents/rucool/rmi-ocean-mixing/files/1km_ctrl/'
save_dir = '/Users/garzio/Documents/rucool/rmi-ocean-mixing/plots'

Then, define some other parameters, such as the geographical extent for plotting the surface map, as well as the variable and height you want to plot. We'll start with plotting wind speed at at height of 160m (which is our theoretical 15MW turbine's hub height). The variable name from the filename is ws, this will be used for a number of different things including finding the correct files, plotting configurations, and naming save file names. A reminder that you can either generate the files yourself or download from [here](https://marine.rutgers.edu/~lgarzio/rmi_ocean_mixing/)). The full list of the variable names that you can plot are:
- HFX: upward heat flux at the surface
- Q2: water vapor mixing ratio at the surface
- T2: air temperature at the surface
- TEMP: air temperature at heights 120m, 160m, 200m, 300m
- TKE_PBL: turbulent kinetic energy at heights 120m, 160m, 200m, 300m
- TSK: surface skin temperature
- UST: friction velocity at the surface
- ws10: wind speed at 10m height
- ws: wind speed at heights 120m, 160m, 200m, 300m

In [45]:
extent = [-75.1, -72.2, 38.1, 40.5]  # [lon_min, lon_max, lat_min, lat_max]
variable = 'ws'  # variable name
h = 160  # height you want to plot

Now, we'll define some plotting configurations for each variable so you don't have to change the code every time you want to plot a different variable. These are the configurations I came up with that most closely align with the plots from Golbazi et al 2022 [DOI 10.1088/1748-9326/ac6e49](https://iopscience.iop.org/article/10.1088/1748-9326/ac6e49/meta). For each variable it specifies colormaps, ranges to mask when plotting the difference plots, colorbar limits and bins for the colorbar.

In [46]:
configs = dict(
        ws=dict(cmap=plt.get_cmap('BuPu'), diffcmap=plt.get_cmap('RdBu_r'), diffmask=[-0.1, 0.1], difflims=[-2, 2], diffbins=40),
        ws10=dict(cmap=plt.get_cmap('BuPu'), diffcmap=plt.get_cmap('RdBu_r'), diffmask=[-0.1, 0.1], difflims=[-2, 2], diffbins=40),
        UST=dict(cmap=plt.get_cmap('YlGnBu'), diffcmap=plt.get_cmap('Spectral_r'), diffmask=[-0.01, 0.01], lims=[0.1, 0.3], bins=20, difflims=[-0.1, 0.1], diffbins=60),
        Q2=dict(cmap=cmo.cm.rain, diffcmap=plt.get_cmap('BrBG'), diffmask=[-0.000025, 0.000025], lims=[0, 0.02], bins=20, difflims=[-0.00015, 0.00015], diffbins=20),
        T2=dict(cmap=cmo.cm.thermal, diffmask=[-0.01, 0.01], difflims=[-0.2, 0.2], diffbins=40),
        TEMP=dict(cmap=cmo.cm.thermal, diffmask=[-0.01, 0.01], difflims=[-0.2, 0.2], diffbins=40),
        TKE_PBL=dict(cmap=cmo.cm.tempo, diffmask=[-0.01, 0.01], lims=[0, 0.2], bins=20, difflims=[-0.12, 0.12], diffbins=40),
        HFX=dict(cmap=cmo.cm.thermal, diffmask=[-0.1, 0.1], lims=[-10, 10], bins=100, difflims=[-1.5, 1.5], diffbins=40),
        TSK=dict(cmap=cmo.cm.thermal, diffmask=[-0.01, 0.01], difflims=[-0.2, 0.2], diffbins=40)
    )

vconfigs = configs[variable] # get the variable specific configurations for the variable you are plotting
print(f'Plotting configs for {variable}: {vconfigs}')

Plotting configs for ws: {'cmap': <matplotlib.colors.LinearSegmentedColormap object at 0x16b57db70>, 'diffcmap': <matplotlib.colors.LinearSegmentedColormap object at 0x16b57d3c0>, 'diffmask': [-0.1, 0.1], 'difflims': [-2, 2], 'diffbins': 40}


Find the correct files based on the variable name you provided, open those files using xarray and look at the contents of the file.

In [47]:
f = f'{file_dir}/ruwrf_1km_wf2km_nyb_{variable}_avg_20220601_20220831.nc'
fc = f'{file_dir_control}/ruwrf_1km_ctrl_{variable}_avg_20220601_20220831.nc'

ds = xr.open_dataset(f)
dsc = xr.open_dataset(fc)

print(ds)

<xarray.Dataset>
Dimensions:   (time: 2207, height: 4, south_north: 399, west_east: 399)
Coordinates:
    XLONG     (south_north, west_east) float32 ...
    XLAT      (south_north, west_east) float32 ...
  * height    (height) int32 120 160 200 300
  * time      (time) datetime64[ns] 2022-06-01T01:00:00 ... 2022-08-31T23:00:00
    time_run  (time) datetime64[ns] ...
Dimensions without coordinates: south_north, west_east
Data variables:
    U         (time, height, south_north, west_east) float32 ...
    V         (time, height, south_north, west_east) float32 ...
    ws        (time, height, south_north, west_east) float32 ...
    ws_avg    (height, south_north, west_east) float32 ...
Attributes: (12/165)
    title:                           Rutgers Weather Research and Forecasting...
    summary:                         Processed netCDF containing subset of RU...
    keywords:                        Weather Advisories > Marine Weather/Fore...
    Conventions:                     CF-1.

If there are multiple heights in the file, there will be a "height" dimension that lists the heights. Grab the model output from the height you want to plot, or if there are not multiple heights, no need to subset the dataset. The snippet of code below accounts for both situations. Also, specify the name of the variable you want to plot (it's just your variable name with "_avg" attached to the end to signify that it's the 3-month summer average). Then, print your dataset again so you can make sure you have the model output from the height you want.

In [48]:
try:
    ds = ds.sel(height=h)
    dsc = dsc.sel(height=h)
except KeyError:
    print('File does not contain height coordinate')

varnameavg = f'{variable}_avg'  # variable name for the average dataset we want to plot

print(ds)

<xarray.Dataset>
Dimensions:   (time: 2207, south_north: 399, west_east: 399)
Coordinates:
    XLONG     (south_north, west_east) float32 ...
    XLAT      (south_north, west_east) float32 ...
    height    int32 160
  * time      (time) datetime64[ns] 2022-06-01T01:00:00 ... 2022-08-31T23:00:00
    time_run  (time) datetime64[ns] ...
Dimensions without coordinates: south_north, west_east
Data variables:
    U         (time, south_north, west_east) float32 ...
    V         (time, south_north, west_east) float32 ...
    ws        (time, south_north, west_east) float32 ...
    ws_avg    (south_north, west_east) float32 ...
Attributes: (12/165)
    title:                           Rutgers Weather Research and Forecasting...
    summary:                         Processed netCDF containing subset of RU...
    keywords:                        Weather Advisories > Marine Weather/Fore...
    Conventions:                     CF-1.4
    naming_authority:                edu.rutgers.marine.rucool

Now we have a smaller subset of the model output at a single height, if that step was necessary for the specific variable you want to plot since some variables are only provided at one height. Next, we're going to subset the model output to the boundaries we specified at the beginning. The function "subset_grid" below will subset our model output. The "data" input to the function is the xarray data array that has latitude and longitude as coordinates. Let's look at that first before subsetting the data array.

In [49]:
ds[varnameavg]

<xarray.DataArray 'ws_avg' (south_north: 399, west_east: 399)>
[159201 values with dtype=float32]
Coordinates:
    XLONG    (south_north, west_east) float32 ...
    XLAT     (south_north, west_east) float32 ...
    height   int32 160
Dimensions without coordinates: south_north, west_east
Attributes:
    units:      m s-1
    comment:    ws average from 20220601 to 20220831
    long_name:  Average Wind Speed

Here you can see that ws_avg is the Average Wind Speed (from 2022-06-01 to 2022-08-31) at height=160m with XLONG and XLAT as coordinates. We can now pass this data array (and the same data array from the control version) through our subset_grid function below to subset our model output.

In [50]:
def subset_grid(ext, data, lon_name, lat_name):
    """
    This function subsets the model output to the defined boundaries
    ext: list of boundaries [lon_min, lon_max, lat_min, lat_max]
    data: xarray data array
    lon_name: name of the longitude coordinate
    lat_name: name of the latitude coordinate
    """
    lonx = data[lon_name]
    laty = data[lat_name]

    lon_idx = np.logical_and(lonx > ext[0], lonx < ext[1])
    lat_idx = np.logical_and(laty > ext[2], laty < ext[3])

    # find i and j indices of lon/lat in boundaries
    ind = np.where(np.logical_and(lat_idx, lon_idx))

    # subset data from min i,j corner to max i,j corner
    # there will be some points outside of defined boundaries because grid is not rectangular
    data_sub = np.squeeze(data)[range(np.min(ind[0]), np.max(ind[0]) + 1), range(np.min(ind[1]), np.max(ind[1]) + 1)]
    lon = data_sub[lon_name]
    lat = data_sub[lat_name]

    return data_sub

In [51]:
vsub = subset_grid(extent, ds[varnameavg], 'XLONG', 'XLAT')
vctrlsub = subset_grid(extent, dsc[varnameavg], 'XLONG', 'XLAT')
print(vsub)

<xarray.DataArray 'ws_avg' (south_north: 269, west_east: 253)>
[68057 values with dtype=float32]
Coordinates:
    XLONG    (south_north, west_east) float32 -75.09 -75.08 ... -72.14 -72.13
    XLAT     (south_north, west_east) float32 38.1 38.1 38.1 ... 40.49 40.49
    height   int32 160
Dimensions without coordinates: south_north, west_east
Attributes:
    units:      m s-1
    comment:    ws average from 20220601 to 20220831
    long_name:  Average Wind Speed


There is our subset model output! You can see the shape of the dataset is smaller (south_north: 269, west_east: 253) compared to before we subset (south_north: 399, west_east: 399).